# GetDataPrescriptionMedicare

This notebook takes the raw data from the Centers for Medicare & Medicaid Services and cleans it. 

In [1]:
%matplotlib inline

from bs4 import BeautifulSoup
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import string
import requests
import time

Read in the data from the raw excel spreadsheet. The sheet labeled "COUNTY" is the correct sheet, and the column rows are in row 3 (0 indexed).

In [2]:
df = pd.read_excel('DataPrescriptionMedicareRaw.xlsx', 'COUNTY', header=3);
print len(df);
df.head()

3077


,State Name,State Abbreviation,State FIPS,County Name,FIPS,Part D Prescribers 2013,Opioid Claims 2013,Overall Claims 2013,Opioid Prescribing Rate 2013,Part D Prescribers 2014,Opioid Claims 2014,Overall Claims 2014,Opioid Prescribing Rate 2014,Percentage Point Difference in Opioid Prescribing Rate
0,Alaska,AK,2,Aleutians East,2013,2,42,690,6.086957,2,83,1543,5.379132,-0.707825
1,Alaska,AK,2,Aleutians West,2016,13,141,2574,5.477855,12,167,2942,5.676411,0.198555
2,Alaska,AK,2,Anchorage,2020,1202,38888,678171,5.734247,1224,45610,755269,6.038908,0.304661
3,Alaska,AK,2,Bethel,2050,48,1329,25906,5.130086,51,1559,25410,6.135380,1.005294
4,Alaska,AK,2,Bristol Bay,2060,1,0,26,0.000000,1,0,27,0.000000,NaN


Next, rename the columns to match those from dataframes containing other data. Also, the difference in prescribing rate is not of use, so drop it.

In [3]:
rename_dict = {'State Name':'State', 'State Abbreviation':'ST',
               'State FIPS':'FIPS State', 'County Name':'County'}
df.rename(columns=rename_dict, inplace=True);
cols_to_drop = ['Percentage Point Difference in Opioid Prescribing Rate'];
df.drop(cols_to_drop, axis=1, inplace=True);
df.head()

,State,ST,FIPS State,County,FIPS,Part D Prescribers 2013,Opioid Claims 2013,Overall Claims 2013,Opioid Prescribing Rate 2013,Part D Prescribers 2014,Opioid Claims 2014,Overall Claims 2014,Opioid Prescribing Rate 2014
0,Alaska,AK,2,Aleutians East,2013,2,42,690,6.086957,2,83,1543,5.379132
1,Alaska,AK,2,Aleutians West,2016,13,141,2574,5.477855,12,167,2942,5.676411
2,Alaska,AK,2,Anchorage,2020,1202,38888,678171,5.734247,1224,45610,755269,6.038908
3,Alaska,AK,2,Bethel,2050,48,1329,25906,5.130086,51,1559,25410,6.135380
4,Alaska,AK,2,Bristol Bay,2060,1,0,26,0.000000,1,0,27,0.000000


The data for the mortality rate has separate entries for each year. Thus, the above dataframe is split (filtered) into two new dataframes for each year, then concatenated. Before concatenation, a year column is added to each dataframe with a single value corresponding to the correct year. Furthermore, the index of the second dataframe is incremented by the length of the first dataframe to make sure all of the entries are unique.

In [4]:
cols_to_copy = ['State', 'ST', 'FIPS State', 'County', 'FIPS',
                'Part D Prescribers 2013', 'Opioid Claims 2013',
                'Overall Claims 2013', 'Opioid Prescribing Rate 2013'];
df_2013 = df[cols_to_copy].copy();
rename_dict = {'Part D Prescribers 2013':'Part D Prescribers',
               'Opioid Claims 2013':'Opioid Claims',
               'Overall Claims 2013':'Overall Claims',
               'Opioid Prescribing Rate 2013':'Opioid Prescribing Rate'};
df_2013.rename(columns=rename_dict, inplace=True);
df_2013['Year'] = 2013;

cols_to_copy = ['State', 'ST', 'FIPS State', 'County', 'FIPS',
                'Part D Prescribers 2014', 'Opioid Claims 2014',
                'Overall Claims 2014', 'Opioid Prescribing Rate 2014'];
df_2014 = df[cols_to_copy].copy();
rename_dict = {'Part D Prescribers 2014':'Part D Prescribers',
               'Opioid Claims 2014':'Opioid Claims',
               'Overall Claims 2014':'Overall Claims',
               'Opioid Prescribing Rate 2014':'Opioid Prescribing Rate'};
df_2014.rename(columns=rename_dict, inplace=True);
df_2014['Year'] = 2014;
df_2014.index = df_2014.index + len(df_2013)

cleandf = pd.concat([df_2013, df_2014]);
del df_2013, df_2014;

print len(cleandf);
cleandf.head()

6154


,State,ST,FIPS State,County,FIPS,Part D Prescribers,Opioid Claims,Overall Claims,Opioid Prescribing Rate,Year
0,Alaska,AK,2,Aleutians East,2013,2,42,690,6.086957,2013
1,Alaska,AK,2,Aleutians West,2016,13,141,2574,5.477855,2013
2,Alaska,AK,2,Anchorage,2020,1202,38888,678171,5.734247,2013
3,Alaska,AK,2,Bethel,2050,48,1329,25906,5.130086,2013
4,Alaska,AK,2,Bristol Bay,2060,1,0,26,0.000000,2013


In [5]:
nullmask = pd.isnull(cleandf);
print np.sum(nullmask);

cleandf.dropna(inplace=True);
print len(cleandf);
cleandf.head()

State                      0
ST                         0
FIPS State                 0
County                     0
FIPS                       0
Part D Prescribers         7
Opioid Claims              7
Overall Claims             7
Opioid Prescribing Rate    7
Year                       0
dtype: int64
6147


,State,ST,FIPS State,County,FIPS,Part D Prescribers,Opioid Claims,Overall Claims,Opioid Prescribing Rate,Year
0,Alaska,AK,2,Aleutians East,2013,2,42,690,6.086957,2013
1,Alaska,AK,2,Aleutians West,2016,13,141,2574,5.477855,2013
2,Alaska,AK,2,Anchorage,2020,1202,38888,678171,5.734247,2013
3,Alaska,AK,2,Bethel,2050,48,1329,25906,5.130086,2013
4,Alaska,AK,2,Bristol Bay,2060,1,0,26,0.000000,2013


Finally, save the cleaned dataframe to a new json file.

In [6]:
cleandf.to_json("DataPrescriptionMedicare.json");

For loading the cleaned data back as a dataframe, the index needs to be converted to a string.

In [7]:
with open('DataPrescriptionMedicare.json', 'r') as f:
    cleandict = json.load(f);

cleandf = pd.DataFrame(cleandict);
cleandf.index = cleandf.index.astype(int);
cleandf.sort_index(inplace=True);
cleandf.head()

,County,FIPS,FIPS State,Opioid Claims,Opioid Prescribing Rate,Overall Claims,Part D Prescribers,ST,State,Year
0,Aleutians East,2013,2,42,6.086957,690,2,AK,Alaska,2013
1,Aleutians West,2016,2,141,5.477855,2574,13,AK,Alaska,2013
2,Anchorage,2020,2,38888,5.734247,678171,1202,AK,Alaska,2013
3,Bethel,2050,2,1329,5.130086,25906,48,AK,Alaska,2013
4,Bristol Bay,2060,2,0,0.000000,26,1,AK,Alaska,2013
